In [12]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy import stats

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("=" * 80)
print("CALIFORNIA HOUSING PRICE PREDICTION - REGRESSION ANALYSIS")
print("=" * 80)

# ---------------------------
# Step 0: dataset path check
# ---------------------------
DATA_PATH = '/content/housing.csv'  # in Colab upload to /content/ or change path

if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"Dataset not found at {DATA_PATH}. Upload housing.csv to this path.")

# ---------------------------
# STEP 1: Load & Inspect
# ---------------------------
df_raw = pd.read_csv(DATA_PATH)
df = df_raw.copy() # Use df for processing to keep df_raw pristine
print(f"Dataset shape: {df.shape}")
print(df.head(3))
print("\nMissing values per column:\n", df.isnull().sum())

# Target stats
print("\nTarget (median_house_value) stats:")
print(df['median_house_value'].describe())

# ---------------------------
# STEP 2: EDA (brief; plots saved)
# ---------------------------
os.makedirs('outputs', exist_ok=True)

# Target histogram
plt.figure()
df['median_house_value'].hist(bins=50)
plt.title('Median House Value Distribution')
plt.xlabel('Median House Value ($)')
plt.ylabel('Count')
plt.savefig('outputs/housing_target_dist.png', dpi=150)
plt.close()

# Geographic scatter colored by target
plt.figure(figsize=(8,6))
plt.scatter(df['longitude'], df['latitude'], c=df['median_house_value'], cmap='viridis', s=8, alpha=0.4)
plt.colorbar(label='House Value')
plt.title('Geographic plot')
plt.savefig('outputs/geographic.png', dpi=150)
plt.close()

# Correlation heatmap of numeric features
num_cols = df.select_dtypes(include=[np.number]).columns
plt.figure(figsize=(10,8))
sns.heatmap(df[num_cols].corr(), annot=True, fmt=".2f", cmap='coolwarm', center=0)
plt.title('Correlation Matrix')
plt.savefig('outputs/corr_matrix.png', dpi=150)
plt.close()

print("EDA plots saved to /content/outputs/")

# ---------------------------
# STEP 3: Preprocessing & Features
# ---------------------------
df_processed = df.copy()

# Missing values
if df_processed['total_bedrooms'].isnull().sum() > 0:
    df_processed['total_bedrooms'].fillna(df_processed['total_bedrooms'].median(), inplace=True)

# Feature engineering
df_processed['rooms_per_household'] = df_processed['total_rooms'] / df_processed['households']
df_processed['bedrooms_per_room'] = df_processed['total_bedrooms'] / df_processed['total_rooms']
df_processed['population_per_household'] = df_processed['population'] / df_processed['households']
df_processed['log_median_income'] = np.log1p(df_processed['median_income'])
df_processed['log_population'] = np.log1p(df_processed['population'])

# Clip engineered extremes (practical)
df_processed['rooms_per_household'] = df_processed['rooms_per_household'].clip(upper=50)
df_processed['population_per_household'] = df_processed['population_per_household'].clip(upper=50)

# One-hot encode ocean_proximity
df_encoded = pd.get_dummies(df_processed, columns=['ocean_proximity'], prefix='ocean')

# Drop any non-numeric leftovers (none expected)
X = df_encoded.drop('median_house_value', axis=1).select_dtypes(include=[np.number])
y = df_encoded['median_house_value']

print("Final feature matrix shape:", X.shape)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ---------------------------
# STEP 4: Models & evaluation
# ---------------------------
def evaluate_model(model, X_tr, X_te, y_tr, y_te):
    model.fit(X_tr, y_tr)
    y_tr_pred = model.predict(X_tr)
    y_te_pred = model.predict(X_te)
    train_rmse = np.sqrt(mean_squared_error(y_tr, y_tr_pred))
    test_rmse = np.sqrt(mean_squared_error(y_te, y_te_pred))
    test_mae = mean_absolute_error(y_te, y_te_pred)
    train_r2 = r2_score(y_tr, y_tr_pred)
    test_r2 = r2_score(y_te, y_te_pred)
    return {
        'Train_RMSE': train_rmse,
        'Test_RMSE': test_rmse,
        'Test_MAE': test_mae,
        'Train_R2': train_r2,
        'Test_R2': test_r2
    }, model

models = {
    'LinearRegression': LinearRegression(),
    'Ridge_a1': Ridge(alpha=1.0, random_state=42),
    'Lasso_a1': Lasso(alpha=1.0, random_state=42),
    'DecisionTree_d5': DecisionTreeRegressor(max_depth=5, random_state=42),
    'DecisionTree_d10': DecisionTreeRegressor(max_depth=10, random_state=42),
    'DecisionTree_full': DecisionTreeRegressor(random_state=42),
    'RandomForest_100': RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42)
}

results = []
trained = {}
for name, m in models.items():
    print("Training:", name)
    res, trained_model = evaluate_model(m, X_train_scaled, X_test_scaled, y_train, y_test)
    res['Model'] = name
    results.append(res)
    trained[name] = trained_model
    print(f"  Train RMSE: ${res['Train_RMSE']:.2f}, Test RMSE: ${res['Test_RMSE']:.2f}, Test MAE: ${res['Test_MAE']:.2f}")

results_df = pd.DataFrame(results).sort_values('Test_RMSE')
results_df.to_csv('outputs/model_comparison.csv', index=False)
print("Model comparison saved to outputs/model_comparison.csv")

# ---------------------------
# STEP 5: Bias-Variance (Decision Tree depth sweep)
# ---------------------------
depths = list(range(1, 21))
train_rmse_list, test_rmse_list = [], []
for d in depths:
    dt = DecisionTreeRegressor(max_depth=d, random_state=42)
    dt.fit(X_train_scaled, y_train)
    train_rmse_list.append(np.sqrt(mean_squared_error(y_train, dt.predict(X_train_scaled))))
    test_rmse_list.append(np.sqrt(mean_squared_error(y_test, dt.predict(X_test_scaled))))

plt.figure()
plt.plot(depths, train_rmse_list, marker='o', label='Train RMSE')
plt.plot(depths, test_rmse_list, marker='s', label='Test RMSE')
plt.xlabel('Tree depth')
plt.ylabel('RMSE ($)')
plt.title('Bias-Variance: Tree Depth vs RMSE')
plt.legend()
plt.grid()
plt.savefig('outputs/bias_variance_analysis.png', dpi=150)
plt.close()

# Best depth by test RMSE
best_depth = depths[np.argmin(test_rmse_list)]
print(f"Best tree depth by test RMSE: {best_depth} with RMSE ${min(test_rmse_list):.2f}")

# ---------------------------
# STEP 6: Learning curve for best model
# ---------------------------
best_model_name = results_df.iloc[0]['Model']
best_model = trained[best_model_name]
print("Generating learning curve for:", best_model_name)

# Use neg_mean_squared_error scorer then sqrt
train_sizes, train_scores, test_scores = learning_curve(
    best_model, X_train_scaled, y_train,
    train_sizes=np.linspace(0.1,1.0,10), cv=5, scoring='neg_mean_squared_error', n_jobs=-1
)
train_rmse_mean = np.sqrt(-train_scores.mean(axis=1))
test_rmse_mean = np.sqrt(-test_scores.mean(axis=1))
plt.figure()
plt.plot(train_sizes, train_rmse_mean, 'o-', label='Train RMSE')
plt.plot(train_sizes, test_rmse_mean, 's-', label='Validation RMSE')
plt.xlabel('Training size')
plt.ylabel('RMSE ($)')
plt.title(f'Learning Curve: {best_model_name}')
plt.legend()
plt.grid()
plt.savefig('outputs/learning_curve.png', dpi=150)
plt.close()
print("Learning curve saved to outputs/learning_curve.png")

# ---------------------------
# STEP 7: Residual analysis for best model
# ---------------------------
y_pred = best_model.predict(X_test_scaled)
residuals = y_test - y_pred

plt.figure(figsize=(8,6))
plt.scatter(y_pred, residuals, s=8, alpha=0.5)
plt.axhline(0, color='r', linestyle='--')
plt.xlabel('Predicted')
plt.ylabel('Residuals')
plt.title('Residuals vs Predicted')
plt.savefig('outputs/residuals_vs_predicted.png', dpi=150)
plt.close()

# ---------------------------
# STEP 8: Feature importance (if RF present)
# ---------------------------
if 'RandomForest_100' in trained:
    rf = trained['RandomForest_100']
    feat_imp = pd.DataFrame({'feature': X.columns, 'importance': rf.feature_importances_}).sort_values('importance', ascending=False)
    feat_imp.head(15).to_csv('outputs/feature_importance_top15.csv', index=False)
    plt.figure(figsize=(8,6))
    top = feat_imp.head(15)
    plt.barh(range(len(top)), top['importance'])
    plt.yticks(range(len(top)), top['feature'])
    plt.gca().invert_yaxis()
    plt.title('Top 15 Feature Importance (RF)')
    plt.savefig('outputs/feature_importance.png', dpi=150)
    plt.close()
    print("Feature importance saved to outputs/feature_importance.png")

# ---------------------------
# STEP 9: Underfitting / Overfitting summary
# ---------------------------
results_df['Error_Gap'] = abs(results_df['Train_RMSE'] - results_df['Test_RMSE'])
underfit_threshold = results_df['Test_RMSE'].quantile(0.75)
overfit_threshold = results_df['Error_Gap'].quantile(0.75)
underfitting = results_df[results_df['Test_RMSE'] > underfit_threshold]
overfitting = results_df[results_df['Error_Gap'] > overfit_threshold]

print("\nUnderfitting candidates:\n", underfitting[['Model','Train_RMSE','Test_RMSE']])
print("\nOverfitting candidates:\n", overfitting[['Model','Train_RMSE','Test_RMSE','Error_Gap']])

# ---------------------------
# STEP 10: Real-world issues (print correctly)
# ---------------------------
real_world_issues = """
1. OUTLIERS: extreme house prices skew RMSE and may require log-scaling or robust losses.
2. NON-LINEARITY: relationships (income vs price) are non-linear; trees/ensembles often perform better.
3. SPATIAL AUTOCORRELATION: geographic clustering suggests spatial CV might be better than random split.
4. MISSING DATA PATTERNS: simple median imputation used; consider modeling missingness if MNAR.
5. DATA AGE: dataset is dated; production models must manage dataset drift.
"""
print(real_world_issues)

print("\nAll outputs saved under /content/outputs/ - download for inclusion in PDF/GitHub.")
print("Done.")

CALIFORNIA HOUSING PRICE PREDICTION - REGRESSION ANALYSIS
Dataset shape: (20640, 10)
   longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0    -122.23     37.88                41.0        880.0           129.0   
1    -122.22     37.86                21.0       7099.0          1106.0   
2    -122.24     37.85                52.0       1467.0           190.0   

   population  households  median_income  median_house_value ocean_proximity  
0       322.0       126.0         8.3252            452600.0        NEAR BAY  
1      2401.0      1138.0         8.3014            358500.0        NEAR BAY  
2       496.0       177.0         7.2574            352100.0        NEAR BAY  

Missing values per column:
 longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: in

In [9]:
# Cell A — Save best model & scaler
import os, joblib

# safe defaults if variables missing
OUT_DIR = globals().get('OUT_DIR', '/content/outputs')
os.makedirs(OUT_DIR, exist_ok=True)

best_model_name = globals().get('best_model_name', None)
trained = globals().get('trained', {})

if best_model_name is None and len(trained) > 0:
    best_model_name = list(trained.keys())[0]

if best_model_name in trained:
    best_model = trained[best_model_name]
    model_path = os.path.join(OUT_DIR, f"{best_model_name}_model.joblib")
    joblib.dump(best_model, model_path)
    print("Saved best model to:", model_path)
else:
    print("Best model not found in session. Available models:", list(trained.keys()))

# Save scaler if present
scaler = globals().get('scaler', None)
if scaler is not None:
    scaler_path = os.path.join(OUT_DIR, "scaler.joblib")
    joblib.dump(scaler, scaler_path)
    print("Saved scaler to:", scaler_path)
else:
    print("No scaler found in session (skipping).")


Saved best model to: /content/outputs/GradientBoosting_model.joblib
Saved scaler to: /content/outputs/scaler.joblib


In [14]:
# Cell B — Build a one-page PDF summary (fpdf)
!pip install -q fpdf

from fpdf import FPDF
import os
import pandas as pd

OUT_DIR = globals().get('OUT_DIR', '/content/outputs')
os.makedirs(OUT_DIR, exist_ok=True)

results_df = globals().get('results_df', None)
df = results_df.copy() if results_df is not None else None

# dataset stats
df_orig = globals().get('df_raw', None) # Correctly reference the original raw DataFrame
count = df_orig.shape[0] if df_orig is not None else 'N/A'
mean_target = df_orig['median_house_value'].mean() if df_orig is not None else 'N/A'

# prepare top 3 models table
top3 = df[['Model','Train_RMSE','Test_RMSE','Test_MAE']].head(3) if df is not None else None

# Under/Overfitting note
note_lines = [
"Linear models show underfitting (high train & test RMSE).",
"Full decision tree shows overfitting (near-zero train RMSE, high test RMSE).",
"Ensembles (RF/GB) balance bias and variance and give best test RMSEs.",
"Consider spatial CV and robust losses in production to handle outliers & spatial bias."
]

pdf_path = os.path.join(OUT_DIR, 'summary_report.pdf')
pdf = FPDF(orientation='P', unit='mm', format='A4')
pdf.set_auto_page_break(auto=True, margin=12)
pdf.add_page()
pdf.set_font("Arial", 'B', 16)
pdf.cell(0, 8, "California Housing - Module 1 (Q2) Summary", ln=1, align='C')
pdf.ln(2)

pdf.set_font("Arial", size=11)
pdf.cell(0, 6, f"Samples: {count}   |   Target mean: ${mean_target:,.2f}", ln=1)
pdf.ln(2)

pdf.set_font("Arial", 'B', 12)
pdf.cell(0, 6, "Top 3 Models (by Test RMSE)", ln=1)
pdf.ln(1)

pdf.set_font("Courier", size=10)
if top3 is not None:
    # header
    pdf.cell(70,6,"Model", border=0)
    pdf.cell(40,6,"Train_RMSE", border=0)
    pdf.cell(40,6,"Test_RMSE", border=0)
    pdf.cell(30,6,"Test_MAE", ln=1, border=0)
    for _, row in top3.iterrows():
        pdf.cell(70,6,str(row['Model'])[:40], border=0)
        pdf.cell(40,6,f"${row['Train_RMSE']:,.0f}", border=0)
        pdf.cell(40,6,f"${row['Test_RMSE']:,.0f}", border=0)
        pdf.cell(30,6,f"${row['Test_MAE']:,.0f}", ln=1, border=0)
else:
    pdf.cell(0,6,"Results table not available.", ln=1)

pdf.ln(4)
pdf.set_font("Arial", 'B', 12)
pdf.cell(0,6,"Underfitting / Overfitting Notes", ln=1)
pdf.set_font("Arial", size=11)
for l in note_lines:
    pdf.multi_cell(0,5," - " + l)
pdf.ln(3)

pdf.set_font("Arial", 'I', 9)
pdf.multi_cell(0,5,"Outputs saved under /content/outputs/. Figures: EDA, bias-variance plot, learning curve, residuals, feature importance. Use GitHub link in submission.")
pdf.output(pdf_path)
print("Created PDF summary:", pdf_path)


Created PDF summary: /content/outputs/summary_report.pdf


In [15]:
# Cell D — Show download links (Colab)
from google.colab import files
print("You can now download these files:")
for fname in ['house_price_outputs.zip', 'outputs/model_comparison.csv', 'outputs/learning_curve.png', 'outputs/bias_variance_analysis.png', 'outputs/feature_importance.png', 'outputs/residuals_vs_predicted.png', 'outputs/housing_target_dist.png', 'outputs/geographic.png', 'outputs/corr_matrix.png', 'outputs/feature_importance_top15.csv', 'outputs/learning_curve_fast.png', 'outputs/summary_report.pdf']:
    if os.path.exists(f"/content/{fname}"):
        print("- /content/" + fname)
    elif os.path.exists(f"/content/outputs/{os.path.basename(fname)}"):
        path = f"/content/outputs/{os.path.basename(fname)}"
        print("- " + path)
    else:
        # some items may be just inside outputs
        pass

# Optionally trigger direct download for the zip and PDF
files.download('/content/house_price_outputs.zip')      # will prompt browser download
files.download('/content/outputs/summary_report.pdf')   # will prompt browser download


You can now download these files:
- /content/house_price_outputs.zip
- /content/outputs/model_comparison.csv
- /content/outputs/learning_curve.png
- /content/outputs/bias_variance_analysis.png
- /content/outputs/feature_importance.png
- /content/outputs/residuals_vs_predicted.png
- /content/outputs/housing_target_dist.png
- /content/outputs/geographic.png
- /content/outputs/corr_matrix.png
- /content/outputs/feature_importance_top15.csv
- /content/outputs/summary_report.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
# Cell E (bash)
# (may be slow: installs texlive; skip if not required)
!apt-get update -qq
!apt-get install -y -qq texlive-xetex texlive-fonts-recommended texlive-generic-recommended
!jupyter nbconvert --to pdf /content/your_notebook.ipynb


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
E: Unable to locate package texlive-generic-recommended
[NbConvertApp] WARNING | pattern '/content/your_notebook.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Appli

In [11]:
# Cell C — Zip outputs folder
import shutil, os
OUT_DIR = globals().get('OUT_DIR', '/content/outputs')
zip_path = '/content/house_price_outputs.zip'
if os.path.exists(zip_path):
    os.remove(zip_path)
shutil.make_archive('/content/house_price_outputs', 'zip', OUT_DIR)
print("Created zip:", zip_path)
print("Files inside outputs (sample):")
print(sorted(os.listdir(OUT_DIR))[:40])


Created zip: /content/house_price_outputs.zip
Files inside outputs (sample):
['GradientBoosting_model.joblib', 'bias_variance_analysis.png', 'corr_matrix.png', 'feature_importance.png', 'feature_importance_top15.csv', 'geographic.png', 'housing_target_dist.png', 'learning_curve.png', 'model_comparison.csv', 'residuals_vs_predicted.png', 'scaler.joblib']
